In [1]:
import openpyxl
import pandas as pd
from natsort import index_natsorted
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
# wenn match schon einmal gemacht wurde
be = pd.read_excel("iii-merge-test-excel.xlsx", sheet_name="III Drucke 1501-1560", dtype={"digi":"bool"}, true_values=["WAHR"], false_values=["FALSCH"])

# erster match
# be = pd.read_excel("Thomschke_III_Drucke_1501-1560.xlsm", sheet_name="III Drucke 1501-1560", dtype="string")
iii = pd.read_csv("../abzug/iii.csv", dtype="string")

In [41]:
len(be.columns)

99

# neuer Ansatz: merge

In [3]:
df = be.merge(iii, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

# idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
df.loc[df["IDN"].isna(), "IDN"] = df.idn
df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

# wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
df.loc[df["idn"].notna(), "digi"] = True
df.loc[df["idn"].isna(), "digi"] = False
df.fillna('', inplace=True)
df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))

# die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
spaltenliste = list(df.columns)[:-21]
# spaltenliste.insert(0,"digi")
# spaltenliste.insert(4, "bbg")
# spaltenliste.insert(7, "signatur_g")

df.to_excel("iii-merge-test.xlsx", index=False, columns=spaltenliste)

In [50]:
df.columns[:-21]

Index(['digi', 'Lfd Nr.', 'AKZ', 'IDN', 'bbg_x', 'Link zum Portal', 'Signatur',
       'signatur_g_x', 'Provenienzmerkmal', 'Material', 'Format',
       'Öffnungswinkel', 'Einschränkungen', 'Glasplatte ', 'Verpackung',
       'Verpackung austauschen ', 'Schadensklasse',
       'notwendige Reparatur(en) vor der Digitalisierung, notwendige Reparatur(en) vor der Digitalisierung',
       'Bemerkungen', 'Fragen/ Hinweise\nan DBSM', 'Fragen an M. Steinberg',
       'nicht\nam Stand-ort', 'Größe ÜF\n(BxH)', 'Breite\n(nur Ausreißer)',
       'Dicke\n(>12 cm)', '12° Format\n(<15 cm)', 'Einband-\nart',
       'Einband über-\nformt (ganz od. teilweise)',
       'Buch bereits restau-riert',
       'hohler/\nfester Rücken (mit Einlage/\nVergoldung?)',
       'Steh-\nkanten\n(bei Perg.)', 'Leder pudert ab/roter Zerfall (extrem)',
       'Einband stark defor-miert', 'Be-schläge bes. auftra-gend',
       'Buch-schließe steif', 'Buch-block Pa./Perg.', 'saures Füll-material',
       'Register-marken', '

# Excel-Dateien mit openpyxl öffnen

In [4]:
wb = openpyxl.load_workbook("iii-merge-test-excel.xlsx")
wb.remove(wb["III Drucke 1501-1560"])
ws = wb.create_sheet("III Drucke 1501-1560", 0)
df = df.astype("object")
for r in dataframe_to_rows(df[spaltenliste], index=False):
    ws.append(r)
ws.add_table(openpyxl.worksheet.table.Table(displayName="Basistabelle", ref=ws.dimensions))
wb.save("iii-merge-test-excel.xlsx")

# Anleitung zur Bearbeitung der zusammengefügten Listen

Die Tabelle enthält ganz vorn die neue Spalte "Digi". Steht dort WAHR, wird das Werk digitalisiert, steht FALSCH, wird es nicht digitalisiert.

Zunächst muss überprüft werden, ob Frau Thomschke Daten in eine Zeile erfasst hat, in der eigentlich angebundene Werke oder sonstige untergeordnete Werke stehen. Diese müssen dann in die Zeile des übergeordneten Satzes verschoben werden, z. B. Qd.

Alle mit FALSCH markierten Zeilen, die angebundene Werke und keine Informationen von Frau Thomschke haben, können gelöscht werden.

Alle mit FALSCH markierte Zeilen, die keine IDN und keine AKZ enthalten, aber eine Signatur, wurden von BE "gefunden" für diese Bände muss ein Datensatz angelegt werden. Wenn das passiert ist, wird die IDN und AKZ in die Tabelle kopiert.









# Kommentare

In [27]:
wb = openpyxl.load_workbook("Thomschke_III_Drucke_1501-1560.xlsm")
# print(wb["III Drucke 1501-1560"]["X198"].comment.text)

for cell in wb["III Drucke 1501-1560"]["X"]:
    if cell.comment:
        wb["III Drucke 1501-1560"].cell(row=cell.row, column=cell.column+1, value=cell.comment.text)


In [28]:
wb.save("iii-merge-test-excel-kommentare.xlsx")

# Abgleich Restaurierungsliste

In [47]:
import glob

dateiliste = glob.glob("../abzug/*.csv")

abzug = pd.DataFrame()

for datei in dateiliste:
    abzug = pd.concat([abzug,pd.read_csv(datei, dtype="string")])

be = pd.read_excel("Restaurierungsbedürftige_Bücher_Gesamt_für_Bettina.xlsx")

df = be.merge(abzug, how="inner", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

# idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
df.loc[df["IDN"].isna(), "IDN"] = df.idn
df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

# wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
df.loc[df["idn"].notna(), "digi"] = True
df.loc[df["idn"].isna(), "digi"] = False
df.fillna('', inplace=True)
df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))

# die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
spaltenliste = list(df.columns)[:-21]
spaltenliste.insert(0,"digi")
spaltenliste.insert(4, "bbg")
spaltenliste.insert(7, "signatur_g")

df.to_excel("restaurierungsliste-merge.xlsx", index=False, columns=spaltenliste)


/home/vrekon/.cache/pypoetry/virtualenvs/vrekon-Woto_OI2-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [43]:
be

,Lfd Nr.,AKZ,IDN,Link zum Portal,Signatur,gehört zu mehr-bändigem Werk,Format,nicht\nam Stand-ort,Größe ÜF\n(BxH),Breite\n(nur Ausreißer),...,Risse am Rand,Fehl-stellen im BB\n(groß),Insekten-fraß (stark),Falten / Knicke,saures /\nbrüchiges Papier,Tinten-/ Farbfraß (akut),Register-marken fragil,Klebe-\nstreifen ablösen,Rest.-Aufwand Buchblock\n(in Std.),Anmerkungen für die Restaurierung am Buchblock
0,16.0,L-1491-327139668,1072233924,https://portal.dnb.de/opac.htm?method=simpleSe...,Bö Ink 16,NaN,bis 35 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19.0,L-1476-315503459,1066973067,https://portal.dnb.de/opac.htm?method=simpleSe...,Bö Ink 18,NaN,bis 42 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2,25.0,L-1491-315500182,1066969841,https://portal.dnb.de/opac.htm?method=simpleSe...,Bö Ink 24,NaN,> 42 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Gelenke innen ggf. offen lassen
3,26.0,L-1497-315499915,1066969590,https://portal.dnb.de/opac.htm?method=simpleSe...,Bö Ink 25,NaN,bis 35 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN
4,28.0,L-1488-315496282,1066965986,https://portal.dnb.de/opac.htm?method=simpleSe...,Bö Ink 27,NaN,bis 35 cm,NaN,NaN,NaN,...,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,137.0,L-1729-155561057,994377584,https://portal.dnb.de/opac.htm?method=simpleSe...,Cb 18,NaN,bis 25 cm,NaN,NaN,QF (22x17),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,158.0,L-1741-176001662,1002008700,https://portal.dnb.de/opac.htm?method=simpleSe...,Cb 175,NaN,bis 25 cm,NaN,NaN,QF (37x23),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686,178.0,L-2013-328326,1044098473,https://portal.dnb.de/opac.htm?method=simpleSe...,Cb 651,NaN,bis 25 cm,NaN,NaN,QF (21x32),...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,nur das Nötigste
687,180.0,L-2013-328325,1044098953,https://portal.dnb.de/opac.htm?method=simpleSe...,Cb 652,NaN,bis 25 cm,NaN,NaN,QF (32x19),...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,3.0,NaN
